In [46]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo 
import CoolProp.CoolProp as CP
from scipy.stats import linregress

# CS5_T1 VALUES
Lf = 50/1000 # mm Length of the core
Vf_cs5 = 6619.3183 * 1e-9 #m³  Volume of the fluid in core
Afs_cs5 = 12365.444 * 1e-6 #m² Interfacial surface area between fluid/wall
Acs_cs5 = Vf_cs5 / Lf
Dh_cs5 = (4*Vf_cs5)/Afs_cs5

#CS7.5_T1 VALUES
Lf = 50/1000 # mm Length of the core
Vf_cs7_5 = 8012.6469 * 1e-9 #m³  Volume of the fluid in core
Afs_cs7_5 = 9101.2586 * 1e-6 #m² Interfacial surface area between fluid/wall
Acs_cs7_5 = Vf_cs7_5 / Lf
Dh_cs7_5 = (4*Vf_cs7_5)/Afs_cs7_5

#CS10_T1 VALUES
Lf = 50/1000 # m Length of the core
Vf_cs10 = 8694.9198 * 1e-9 #m³  Volume of the fluid in core
Afs_cs10 = 7726.8569 * 1e-6 #m² Interfacial surface area between fluid/wall
Acs_cs10 = Vf_cs10 / Lf
Dh_cs10 = (4*Vf_cs10)/Afs_cs10

# DEFINING REYNOLDS AND VELOCITY
T = 300
mu = CP.PropsSI('V', 'T', T, 'P', 101325, 'Water')
rho = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Water')


def Reynolds(mfr, dh, acs, mu):
    Re = (mfr * dh) / (acs * mu)
    return Re 

def Reynolds(rho, v, dh, mu):
    Re = (rho * v * dh) / (mu)
    return Re 

def Velocity(mfr, rho, acs):
    Vel = (mfr) / (rho * acs)
    return Vel

def ffactor(dp, dh, L, rho, V):
    f = (dp * dh) / (2*L*rho*(np.power(V,2)))
    return f



In [47]:
col = [9, 8, 10]

df_cs10 = []

for freq in range(25, 55, 5):
    sheetname = f'cs10_t1_coriolisSB_{freq}Hz'
    df = pd.read_excel('cs10_t1_coriolisSB.xlsx', sheetname)
    df = df.iloc[:, col].values
    df_cs10.append(df)

mfr_cs10 = []
dp_cs10 = []
abs_cs10 = []

for df in df_cs10:
    massflowrate_cs10 = df[:, 0]
    differencial_cs10 = df[:, 1] * 1e5
    absolute_cs10 = df[:, 2]
    lastmfr_cs10 = massflowrate_cs10[-100:]
    lastdp_cs10 = differencial_cs10[-100:]
    lastabs_cs10 = absolute_cs10[-100:]        
    
    meanmfr_cs10 = lastmfr_cs10.mean()
    meandp_cs10 = lastdp_cs10.mean()
    meanabs_cs10 = lastabs_cs10.mean() 

    mfr_cs10.append(meanmfr_cs10)
    dp_cs10.append(meandp_cs10)
    abs_cs10.append(meanabs_cs10)

#Reynolds and velocity values 

mfr_cs10 = np.array(mfr_cs10)




V_cs10 = np.array(Velocity(mfr_cs10, rho, Acs_cs10))
Re_cs10_t1 = Reynolds(1000, V_cs10, Dh_cs10, mu)


In [48]:
print(dp_cs10)

[11640.47, 17477.91, 24253.63, 32074.14, 41986.42, 52547.85]


In [49]:
# PLOTS DP x MASS FLOW RATE

trace1 = go.Scatter(

    x=mfr_cs10,
    y=dp_cs10,
    name = 'cs10_t1',
    mode='markers',
    marker=dict(
        size=12,
        color='Red'
    )
)

layout = go.Layout(title = 'Pressure Drop x Mass Flow Rate',
                   xaxis = {'title': 'Mass Flow Rate (kg/s)'},
                   yaxis = {'title': 'Pressure Drop (Bar)'},
                   hovermode = 'closest')
data = [trace1]

fig = go.Figure(data = data, layout = layout)
pyo.plot(fig)

'temp-plot.html'

In [50]:
print(Re_cs10_t1)

[1883.09672387 2339.86960609 2801.99688795 3273.86552646 3774.25851222
 4279.0110745 ]


In [51]:
# PLOTS DP x REYNOLDS

trace1 = go.Scatter(

    x=Re_cs10_t1,
    y=dp_cs10,
    name = 'cs10_t1',
    mode='markers',
    marker=dict(
        size=12,
        color='Red'
    )
)

layout = go.Layout(title = 'Pressure Drop x Mass Flow Rate',
                   xaxis = {'title': 'Reynolds'},
                   yaxis = {'title': 'Pressure Drop (Bar)'},
                   hovermode = 'closest')
data = [trace1]

fig = go.Figure(data = data, layout = layout)
pyo.plot(fig)

'temp-plot.html'

In [52]:

f_cs10 = ffactor(np.array(dp_cs10), Dh_cs10, Lf, rho, V_cs10)


trace1 = go.Scatter(
    x=Re_cs10_t1,
    y=f_cs10,
    name = 'cs5_t1',
    mode='markers',
    marker=dict(
        size=12,
        color='Red'
    )
)

layout = go.Layout(title = 'f x Reynolds',
                   xaxis = {'title': 'Reynolds'},
                   yaxis = {'title': 'f'},
                   hovermode = 'closest')
data = [trace1]

fig = go.Figure(data = data, layout = layout)
pyo.plot(fig)

'temp-plot.html'